In [25]:
import pandas as pd
from datetime import datetime, timedelta

from prophet import Prophet  # https://medium.com/data-folks-indonesia/installing-fbprophet-prophet-for-time-series-forecasting-in-jupyter-notebook-7de6db09f93e
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)


### Read data from csv file

In [26]:
df = pd.read_csv("hourlyMB.csv")
df['report_date'] = pd.to_datetime(df['report_date'])
df.sort_values(by=['report_date'], ascending = True, inplace = True)
df.tail()

,report_date,Technology,Total_DL_MB,Total_UL_MB
4315,2022-01-08 01:00:00,CDMA,23860.08056,4428.123791
4316,2022-01-08 02:00:00,CDMA,20512.50084,3521.416631
4317,2022-01-08 03:00:00,CDMA,19542.88953,3224.817239
4318,2022-01-08 04:00:00,CDMA,17644.54676,3049.085120
4319,2022-01-08 05:00:00,CDMA,16739.64853,2988.817181


### Downlink Data plots

In [27]:
# DL plots
fig = px.line(df,x='report_date',y='Total_DL_MB',color='Technology', title='DL data for last 30 days').update_traces(connectgaps=False)
# connectgaps=False is so teh plot makes it obvious when we are missing data
fig.show()

In [28]:
fig = px.line(df,x='report_date',y='Total_DL_MB',color='Technology', log_y=True, title='DL data for last 30 days, log scale').update_traces(connectgaps=False)
fig.show()

### Uplink Data

In [29]:
#UL plots
fig = px.line(df,x='report_date',y='Total_UL_MB',color='Technology', title='UL data for last 30 days').update_traces(connectgaps=False)
fig.show()

fig = px.line(df,x='report_date',y='Total_UL_MB',color='Technology', log_y=True, title='UL data for last 30 days, log scale').update_traces(connectgaps=False)
fig.show()

### Prophet analysis
https://facebook.github.io/prophet/docs/quick_start.html#python-api
https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/

In [33]:
# Prophet part
def anom_func(row):  # see if data is outside the confidence intervals (above yhat_upper or below yhat_lower)
    if row['y'] > row['yhat_upper'] or row['y'] < row['yhat_lower']:
        return 'anomaly'
    return ''

for tech in df['Technology'].unique():  # loop through the different technologies
    df2 = df[df['Technology'] == tech]  # filter down to just the current technology
    df2 = df2[['report_date','Total_DL_MB']]  # prophet wants just two columns, ds for the time and y for the data
    df2.columns = ['ds', 'y']
    m = Prophet(changepoint_prior_scale=0.01,interval_width = 0.95).fit(df2)  # make prophet object and fit to data
    future = m.make_future_dataframe(periods=24, freq='H')  # make the future dataframe for the predictions
    fcst = m.predict(future)  # predict mean and confidence intervals
    df2['ds'] = pd.to_datetime(df2['ds'])
    df2 = df2.join(fcst.set_index('ds'), on='ds')  # Make df2 have the forecast columns as well as the original data

    df2['anomaly'] = df2.apply(anom_func,axis=1)  # see if actual data is outside cofidence interval
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df2['ds'], y=df2['yhat_upper'],line_color='lightskyblue', name='Upper CI')) # upper confidence interval
    fig.add_trace(go.Scatter(x=df2['ds'], y=df2['yhat_lower'], fill='tonexty',line_color='lightskyblue', name='Lower CI')) # fill to yhat_upper
    fig.add_trace(go.Scatter(x=df2['ds'], y=df2['y'], line_color='black', name= tech + ' hourly'))  # actual data points
    # plot red points for where the data is outside predicted confidence interval
    fig.add_trace(go.Scatter(x=df2[df2['anomaly'] == 'anomaly']['ds'], y=df2[df2['anomaly'] == 'anomaly']['y'],mode='markers', name='Anomaly', marker_color='red'))
    fig.update_layout(
    title= tech + " Data",
    xaxis_title="Date",
    yaxis_title="Hourly MB")
    fig.show()
    count = df2.iloc[-48:][df2.iloc[-48:]['anomaly'] == 'anomaly']['anomaly'].count()
    if count > 4:  # if more than four data points outside confidence interval, print the count.
        print("Anomaly with" + tech + " MB's.  Count: ", count)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Anomaly withALU_FDD MB's.  Count:  6


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
